In [ ]:
!pip install openai-whisper transformers pytube pydub torch mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 14.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvi

In [ ]:
!apt-get -y install mysql-server

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
mysql-server is already the newest version (8.0.37-0ubuntu0.22.04.3).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
!service mysql start

 * Starting MySQL database server mysqld
   ...done.


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
mysql-client is already the newest version (8.0.37-0ubuntu0.22.04.3).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Enter password: 
ERROR 2003 (HY000): Can't connect to MySQL server on '49.37.212.106:3306' (110)


In [ ]:

import mysql.connector
from mysql.connector import Error

try:
    # Replace these with your MySQL database credentials
    connection = mysql.connector.connect(
        host='localhost',
        user='root',
        password='root'
    )

    if connection.is_connected():
        db_Info = connection.get_server_info()
        print(f"Connected to MySQL Server version {db_Info}")

        cursor = connection.cursor()
        cursor.execute("CREATE DATABASE IF NOT EXISTS library")
        print("Database created successfully")
        cursor.execute("USE library")
        print("Database selected successfully")
        cursor.execute("CREATE TABLE IF NOT EXISTS books (id INT AUTO_INCREMENT PRIMARY KEY, title VARCHAR(255), author VARCHAR(255), genre VARCHAR(255))")
        print("Table created successfully")


except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if 'connection' in locals() or 'connection' in globals():
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")


Connected to MySQL Server version 8.0.37-0ubuntu0.22.04.3
Database created successfully
Database selected successfully
Table created successfully
MySQL connection is closed


In [ ]:
import mysql.connector
from mysql.connector import Error

try:
    # Replace 'your-public-ip' with your local machine's public IP address
    connection = mysql.connector.connect(
        host='49.37.212.106',
        user='root',
        password='1234',
        connection_timeout=10  # 10 seconds timeout
    )

    if connection.is_connected():
        db_Info = connection.get_server_info()
        print(f"Connected to MySQL Server version {db_Info}")

        cursor = connection.cursor()
        cursor.execute("CREATE DATABASE IF NOT EXISTS library")
        print("Database created successfully")
        cursor.execute("USE library")
        print("Database selected successfully")
        cursor.execute("CREATE TABLE IF NOT EXISTS books (id INT AUTO_INCREMENT PRIMARY KEY, title VARCHAR(255), author VARCHAR(255), genre VARCHAR(255))")
        print("Table created successfully")

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if 'connection' in locals() or 'connection' in globals():
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")


Error while connecting to MySQL 2003 (HY000): Can't connect to MySQL server on '49.37.212.106:3306' (110)


In [ ]:
import whisper
from pydub import AudioSegment
import os
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import math
from google.colab import files
from pytube import YouTube
import mysql.connector
from datetime import datetime

# Function to convert audio file to WAV format if it's not already in that format
def convert_to_wav(audio_file):
    file_name, file_extension = os.path.splitext(audio_file)
    if file_extension.lower() != '.wav':
        audio = AudioSegment.from_file(audio_file)
        wav_file = f"{file_name}.wav"
        audio.export(wav_file, format="wav")
        return wav_file
    return audio_file

# Function to transcribe audio using Whisper
def transcribe_audio_with_whisper(audio_file):
    # Load the Whisper model
    model = whisper.load_model("base")

    # Convert the audio file to WAV format if necessary
    wav_file = convert_to_wav(audio_file)

    # Transcribe the audio file
    result = model.transcribe(wav_file)

    return result["text"]

# Function to summarize text using Pegasus-XSUM
def summarize_text_with_pegasus(text):
    # Load the Pegasus tokenizer and model
    model_name = "google/pegasus-xsum"
    tokenizer = PegasusTokenizer.from_pretrained(model_name)
    model = PegasusForConditionalGeneration.from_pretrained(model_name)

    # Tokenize the input text
    inputs = tokenizer(text, truncation=True, padding="longest", return_tensors="pt")

    # Calculate appropriate min_length and max_length values
    total_words = len(inputs["input_ids"][0])
    min_length = max(math.ceil(total_words / 5), 30)  # Minimum summary length as 1/5 of total words or 30 tokens, whichever is greater
    max_length = min(math.ceil(total_words / 3), 120)  # Maximum summary length as 1/3 of total words or 120 tokens, whichever is smaller

    # Generate the summary
    summary_ids = model.generate(
        inputs.input_ids,
        num_beams=4,
        min_length=min_length,
        max_length=max_length,
        early_stopping=True
    )
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

# Function to download audio from YouTube
def download_audio_from_youtube(url):
    try:
        yt = YouTube(url)
        audio_stream = yt.streams.filter(only_audio=True).first()
        output_file = audio_stream.download(filename="downloaded_audio")
        return output_file
    except Exception as e:
        print("The video is age-restricted and cannot be accessed or downloaded.")
        return None

# Function to extract audio metadata
def get_audio_metadata(audio_file):
    audio = AudioSegment.from_file(audio_file)

    # Duration in seconds
    duration = len(audio) / 1000.0

    # Bitrate in kbps
    bitrate = audio.frame_rate

    # Sample rate in Hz
    sample_rate = audio.frame_rate

    # Recorded time (assuming current time for this example)
    recorded_time = datetime.now()

    return duration, bitrate, sample_rate, recorded_time

# Function to store data in MySQL
def store_audio_data(audio_file_path, transcription, summary, duration, bitrate, sample_rate, recorded_time):
    try:
        # Connect to MySQL
        conn = mysql.connector.connect(
        host='localhost',
        user='root',
        password='root'
        )

        if conn.is_connected():
            # Check if table exists, create if not
            cursor = conn.cursor()
            cursor.execute("CREATE DATABASE IF NOT EXISTS Audio")
            cursor.execute("USE Audio")
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS audio_data (
                    sno INT AUTO_INCREMENT PRIMARY KEY,
                    audio_file_path VARCHAR(255) NOT NULL,
                    transcription TEXT,
                    summary TEXT,
                    duration FLOAT,
                    bitrate INT,
                    sample_rate INT,
                    recorded_time DATETIME
                )
            """)

            # Insert data into table
            insert_query = "INSERT INTO audio_data (audio_file_path, transcription, summary, duration, bitrate, sample_rate, recorded_time) VALUES (%s, %s, %s, %s, %s, %s, %s)"
            data = (audio_file_path, transcription, summary, duration, bitrate, sample_rate, recorded_time)
            cursor.execute(insert_query, data)

            # Commit changes
            conn.commit()
            print("Data stored successfully in MySQL!")

    except mysql.connector.Error as error:
        print(f"Error storing data in MySQL: {error}")

    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()

# Function to handle YouTube URL
def handle_audio_input_youtube(input_path_or_url):
    if input_path_or_url.startswith("http"):
        # It's a YouTube URL
        audio_file = download_audio_from_youtube(input_path_or_url)
        if audio_file is not None:
            try:
                # Transcribe the audio file
                transcription = transcribe_audio_with_whisper(audio_file)

                # Summarize the transcription
                summary = summarize_text_with_pegasus(transcription)

                # Get audio metadata
                duration, bitrate, sample_rate, recorded_time = get_audio_metadata(audio_file)

                # Store in MySQL
                store_audio_data(audio_file, transcription, summary, duration, bitrate, sample_rate, recorded_time)

                # Print the transcription and summary
                print("Transcription:\n", transcription)
                print("\nSummary:\n", summary)

            except Exception as e:
                print(f"Error processing the audio file: {e}")

# Function to handle local file upload
def handle_audio_input_local():
    uploaded = files.upload()
    for filename in uploaded.keys():
        audio_file = filename
        audio_file = convert_to_wav(audio_file)
        if audio_file is not None:
            try:
                # Transcribe the audio file
                transcription = transcribe_audio_with_whisper(audio_file)

                # Summarize the transcription
                summary = summarize_text_with_pegasus(transcription)

                # Get audio metadata
                duration, bitrate, sample_rate, recorded_time = get_audio_metadata(audio_file)

                # Store in MySQL
                store_audio_data(audio_file, transcription, summary, duration, bitrate, sample_rate, recorded_time)

                # Print the transcription and summary
                print("Transcription:\n", transcription)
                print("\nSummary:\n", summary)

            except Exception as e:
                print(f"Error processing the audio file: {e}")



# Determine if it's a YouTube URL or local file upload
temp = input("Is it a YouTube URL? (y/n): ").strip()
if temp.lower() == "y":
    input_path_or_url = input("Enter the YouTube URL: ").strip()
    handle_audio_input_youtube(input_path_or_url)
else:
    handle_audio_input_local()


Is it a YouTube URL? (y/n): y
Enter the YouTube URL: https://youtu.be/aO1-6X_f74M?si=9uWh7LUCJ9yWJ1zv


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Data stored successfully in MySQL!
Transcription:
  How to Talk about Home Town I live in a city called London. This is not London, England. It is a city in Canada. Some of Canada's cities have the same names as English cities. London, Canada has many restaurants and cafes. For entertainment, you can go to the movie theatre or art gallery. There is a famous university in London called Western University. It is not close to the ocean or lake, so there are no beaches. It is a multicultural city, so there are churches, temples and mosques. I've been living in London for ten years. My favorite part of this city is the nature. There are many parks and forests where you can go for walks, runs, or have picnics with friends. The worst thing about London is that it is far away from other big cities in Canada. You need to drive for almost two hours to get to Toronto or Niagara Falls. This isolation isn't all bad, however. If you drive half an hour outside the city, you can find beautiful quiet f